# Imports

In [3]:
import numpy as np
import data

# Reading Data

In [4]:
dataLoader = data.DataLoader()

pure_dataset = dataLoader.get_pure_data()
solution_a_dataset = dataLoader.get_modified_data_solution_a()
solution_b_dataset_before_projection = dataLoader._get_modified_data_solution_b_before_projection()
solution_b_dataset = dataLoader.get_modified_data_solution_b()

print(f"Pure Dataset Dims:                           {pure_dataset.shape}")
print(f"Solution A Dataset Dims:                     {solution_a_dataset.shape}")
print(f"Solution B (before projection) Dataset Dims: {solution_b_dataset_before_projection.shape}")
print(f"Solution B (after projection) Dataset Dims:  {solution_b_dataset.shape}")

Pure Dataset Dims:                           (9120, 125, 45)
Solution A Dataset Dims:                     (9120, 45)
Solution B (before projection) Dataset Dims: (9120, 5625)
Solution B (after projection) Dataset Dims:  (9120, 654)
